This notebook gives some fundamentals on Bagging and Boosting methods for the star/galaxy/quasar classification.

Written with ❤️ by Viviana Acquaviva for the LSST Data Science Fellowship lecture series.

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

We follow the same steps as before to derive the final data set. Note - because we are dealing with more complex models, I am reducing the sample size to ~10,000 (feel free to change frac to 0.05 if needed).

In [2]:
data = pd.read_csv('star_classification.csv', sep = ',')

df = data.sample(frac = 0.1, random_state = 11)

df.reset_index(drop = True, inplace=True)

In [3]:
seldf = df[['u','g','r','i','z']]

In [4]:
for col in seldf.columns:
    seldf = seldf[seldf[col] >= 0]

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
le = LabelEncoder() #turns categorical into 1 ... N

In [7]:
y = le.fit_transform(df['class'].values)

In [8]:
le.classes_

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [9]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'GALAXY': 0, 'QSO': 1, 'STAR': 2}


In [10]:
target = y[seldf.index]

### Random Forest

In [11]:
model = RandomForestClassifier()

In [12]:
cv = KFold(n_splits=5, random_state=10, shuffle= True)

We can establish a benchmark, as usual:

In [13]:
scores = cross_validate(model, seldf, target, cv = cv, return_train_score = True)

In [14]:
print("Time: %.3fs, Mean train score: %.3f, Mean test score: %.3f, std: %.3f"%(scores['score_time'].mean(), scores['train_score'].mean(), scores['test_score'].mean(), scores['test_score'].std()))

Time: 0.028s, Mean train score: 1.000, Mean test score: 0.843, std: 0.008


As usual, we can get a landscape of model parameters like this:

In [15]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

It may be useful to distinguish between the parameters of the decision tree, the randomization process, and the averaging process.

#### Tree Parameters

Some useful parameters associated to a tree are:

- the minimum number of instances in a leaf node; 
- the minimum number of instances required in a split node;
- the maximum depth of tree;
- the minimum information gain required to decide whether a split is "worth it".

#### Randomization Parameters

Here we find:

-    The number of k < n features that are used in building trees (max_features);

-    The re-sampling (boostrap) of the data set (T or F).

#### Forest Parameters

The number of trees in the forest (n_estimators) can be adjusted, with the general understanding that more trees are better, but at some point performance will plateau, so one can find the trade-off between having more trees and lower runtime.



### Optimizing hyperparameters

Finally, the cell below shows how to run a Grid Search for the parameters for some example sets; if a Grid Search is too expensive, we can also run a Randomized Search, which doesn't guarantee that we'll find the best model, but usually gets us close enough.

In [33]:
#Takes ~ 25 secs min for me

parameters = {'min_impurity_decrease':[0.0, 0.001, 0.01], 'max_depth': [10, 15, 20, 25, 30]}

model = GridSearchCV(RandomForestClassifier(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = -1, return_train_score=True)
model.fit(seldf,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best params, best score: 0.8438 {'max_depth': 30, 'min_impurity_decrease': 0.0}


We can save the results in a data frame, and look at the best models to build some intution.

In [34]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)


In [35]:
scoresCV

,params,mean_test_score,std_test_score,mean_train_score
12,"{'max_depth': 30, 'min_impurity_decrease': 0.0}",0.843783,0.005689,0.999925
9,"{'max_depth': 25, 'min_impurity_decrease': 0.0}",0.843484,0.004941,0.999900
6,"{'max_depth': 20, 'min_impurity_decrease': 0.0}",0.842884,0.004794,0.996950
3,"{'max_depth': 15, 'min_impurity_decrease': 0.0}",0.839483,0.007194,0.975098
0,"{'max_depth': 10, 'min_impurity_decrease': 0.0}",0.822281,0.006079,0.898490
10,"{'max_depth': 25, 'min_impurity_decrease': 0.001}",0.798678,0.010351,0.816207
13,"{'max_depth': 30, 'min_impurity_decrease': 0.001}",0.797778,0.010016,0.814956
7,"{'max_depth': 20, 'min_impurity_decrease': 0.001}",0.797578,0.009856,0.814381
1,"{'max_depth': 10, 'min_impurity_decrease': 0.001}",0.797178,0.011232,0.813706
4,"{'max_depth': 15, 'min_impurity_decrease': 0.001}",0.796878,0.011806,0.814206


Q: what parameters seem to be consistently important for the top models? And which ones are not important?

In [19]:
#min_impurity_decrease = 0 seems quite important
#max_depth = 10 seems quite important

# max_features and n_estimators seem unimportant






For comparison, this is how to set up a Randomized Search:

In [20]:
parameters = {'min_impurity_decrease':[0.5, 0.2, 0.0], \
              'max_features':[None,4,2], 'n_estimators':[10, 50, 100], 'max_depth': [5,10]}

model = RandomizedSearchCV(RandomForestClassifier(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = -1, return_train_score=True, n_iter = 30)
model.fit(seldf,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best params, best score: 0.8278 {'n_estimators': 100, 'min_impurity_decrease': 0.0, 'max_features': 4, 'max_depth': 10}


#### Q: What is the most convenient strategy here?

##### random search slightly quicker and almost same result for best_score. so if you're repeating initialization then maybe random search is good, but grid search seems more robust.

### XGBoost

Medium article explaining XGBoost: [here](https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7); some nice tutorials from XGBoost's site: [here](https://xgboost.readthedocs.io/en/latest/tutorials/index.html)

In [21]:
model = xgb.XGBClassifier()

In [22]:
model.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

This is one possible grid (note - this takes a few minutes on my machine)!

In [23]:
parameters = {'max_depth': [5,10], 'n_estimators':[10, 50,100], 
              'learning_rate': [0.05, 0.1, 0.2], 'subsample':[0.5,1]}

model = GridSearchCV(xgb.XGBRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)

model.fit(seldf,target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

[CV] END max_depth=5, max_features=None, min_impurity_decrease=0.5, n_estimators=50; total time=   0.3s
[CV] END max_depth=5, max_features=None, min_impurity_decrease=0.2, n_estimators=10; total time=   0.1s
[CV] END max_depth=5, max_features=None, min_impurity_decrease=0.2, n_estimators=50; total time=   0.3s
[CV] END max_depth=5, max_features=None, min_impurity_decrease=0.2, n_estimators=100; total time=   0.6s
[CV] END max_depth=5, max_features=None, min_impurity_decrease=0.0, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, max_features=4, min_impurity_decrease=0.0, n_estimators=10; total time=   0.2s
[CV] END max_depth=5, max_features=4, min_impurity_decrease=0.0, n_estimators=50; total time=   0.8s
[CV] END max_depth=5, max_features=2, min_impurity_decrease=0.5, n_estimators=10; total time=   0.0s
[CV] END max_depth=5, max_features=2, min_impurity_decrease=0.5, n_estimators=50; total time=   0.1s
[CV] END max_depth=5, max_features=2, min_impurity_decrease=0.5, n_estimat

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best params, best score: 0.5786 {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.5}


### Task: 

### Try to improve on our current scores, by further optimizing a RF or XGB, and/or by engineering new features that could be helpful.

### Feature importance

Tree-based methods have a built-in feature importance method. It is important that the estimator has been "fit" before you can access the feature importances. If you are using this just to build an understanding of which features most participate to a prediction (and not to assess the performance of a model with a given set of features), you can use the full learning set.

Note:

The ranking (more in general, the information gain associated to each feature) is algorithm-dependent and often affected by correlations among variables. I would encourage you to think about it as an indication, and/or to derive it for a few different algorithms to increase robustness.


In [24]:
model

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    in...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=4,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [5, 10], 'n_estimators': [10, 50, 100],
                         'subsample': [0.5, 1]},
             return_train_score=True, verbose=2)

In [25]:
model.fit(seldf, target)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    in...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=4,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [5, 10], 'n_estimators': [10, 50, 100],
                         'subsample': [0.5, 1]},
             return_train_score=True, verbose=2)

In [26]:
model.feature_importances_

AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [ ]:
importances = model.feature_importances_

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(seldf.shape[1]):
    print("%d. feature: %s, %d (%f)" % (f + 1, seldf.columns[indices[f]], indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(16,6))
plt.title("Feature importances")
plt.bar(range(seldf.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(seldf.shape[1]), seldf.columns[indices])
plt.xlim([-1, seldf.shape[1]])
plt.show()